In [122]:
%pylab inline
import os
import numpy as np
import pandas as pd
from scipy.misc import imread
from sklearn.metrics import accuracy_score
from sklearn.cross_validation import train_test_split
import re

import tensorflow as tf
import keras

from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Convolution2D, Flatten, MaxPooling2D, Reshape, InputLayer
from keras.optimizers import SGD

root_dir = os.path.abspath('/Users/dcolton/Desktop/checked/')
os.path.exists(root_dir)

seed = 128
rng = np.random.RandomState(seed)

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


In [3]:
train = pd.read_csv(os.path.join(root_dir, 'train.csv'))
sample_submission = pd.read_csv(os.path.join(root_dir, 'train.csv'))

In [4]:
dfpngdf = pd.read_csv('checked/pngdf.csv', sep='\t', header=None)
dfb = (pd.DataFrame(dfpngdf[0].map(lambda x: x[:-4])))
df = pd.concat([dfpngdf, dfb], axis = 1)
df.columns = ['png','Title']

In [5]:
namecol = []

for t in df.Title:
    i = t[-3:]
    m = re.match(".+(-\d{1,2})",i)
    if m:
        t = t.replace(m.group(1),"")
    namecol.append(t)
print len(namecol)
        
        
    
    

2824


In [6]:
df.shape

(2824, 2)

In [7]:
df['Stripped']=np.array(namecol)

In [8]:
train['Stripped']=train['Title']

In [9]:
dfnew = pd.merge(df, train, how='left',on=['Stripped'])

In [31]:
dfnew.shape

(3088, 14)

In [11]:
null_data = dfnew[dfnew.isnull().any(axis=1)]

In [12]:
dfnew.drop(null_data.index, inplace=True)

In [20]:
dfnew['label'] = dfnew['Unnamed: 9']
del dfnew['Unnamed: 9']

In [89]:
dfnew['label'].value_counts()

1.0     1388
3.0      719
8.0      407
4.0      263
12.0     226
6.0       65
10.0       9
2.0        8
11.0       3
Name: label, dtype: int64

In [43]:
temp = []
i = 0
for img_name in dfnew['png']:
    for i in xrange(15):
        if os.path.isfile('/Users/dcolton/Desktop/checked/'+ img_name + '-' + str(i)):
            image_path = os.path.join('/Users/dcolton/Desktop/checked',img_name)
            img = imread(image_path, flatten=True)
            img = img.astype('float32')
            temp.append(img)
            i += 1
    if os.path.isfile('/Users/dcolton/Desktop/checked/'+ img_name):
        image_path = os.path.join('/Users/dcolton/Desktop/checked',img_name)
        img = imread(image_path, flatten=True)
        img = img.astype('float32')
        temp.append(img)

In [37]:
dfnew['png'].shape

(3088,)

In [101]:
temp[0]

array([[ 255.,  255.,  255., ...,  255.,  255.,  255.],
       [ 255.,  255.,  255., ...,  255.,  255.,  255.],
       [ 255.,  255.,  255., ...,  255.,  255.,  255.],
       ..., 
       [ 255.,  255.,  255., ...,    0.,    0.,    0.],
       [ 255.,  255.,  255., ...,  255.,  255.,  255.],
       [ 255.,  255.,  255., ...,  255.,  255.,  255.]], dtype=float32)

In [111]:
y = dfnew['label'].values

In [112]:
y

array([ 6.,  4.,  8., ...,  1.,  3.,  1.])

In [107]:
x=np.stack(temp)
x = x.reshape(3088, 90000).astype('Float32')
x /= 255.0

In [108]:
print x.shape
print y.shape

(3088, 90000)
(3088, 13)


In [113]:
xtr, xte, ytr, yte = train_test_split(x, y, stratify=y)

In [114]:
ytr = keras.utils.np_utils.to_categorical(ytr, 13)
yte = keras.utils.np_utils.to_categorical(yte, 13)

In [127]:
input_num_units = 90000
hidden_num_units = 20
output_num_units = 13
epochs = 50
batch_size = 128
pixel_count = 90000
language_count = 13

In [128]:
model = Sequential()
model.add(Dense(128, input_shape=(pixel_count,)))
model.add(Activation('sigmoid'))
# model.add(Dropout(0.5))
model.add(Dense(language_count))
model.add(Activation('softmax'))

sgd = SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd,  metrics=['accuracy'])


# input_reshape = (300,300,1)
# pool_size = (2, 2)
# train_x_temp = xtr.reshape(-1, 300, 300, 1)
# val_x_temp = xte.reshape(-1, 300, 300, 1)


# model = Sequential([
#  InputLayer(input_shape=input_reshape),

#  Convolution2D(25, 5, 5, activation='relu'),
#  MaxPooling2D(pool_size=pool_size),

# #  Convolution2D(25, 5, 5, activation='relu'),
# #  MaxPooling2D(pool_size=pool_size),

# #  Convolution2D(25, 4, 4, activation='relu'),

#  Flatten(),

# #  Dense(output_dim=output_num_units, activation='relu'),
# # 
#  Dense(output_dim=12, input_dim=hidden_num_units, activation='softmax'),
# ])

# model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# trained_model_conv = model.fit(train_x_temp, ytr)
# #         nb_epoch=epochs, batch_size=batch_size, validation_data=(val_x_temp, yte))

In [129]:
trained_model = model.fit(xtr, ytr, epochs=epochs, batch_size=batch_size, validation_data=(xte, yte))

Train on 2316 samples, validate on 772 samples
Epoch 1/50
2316/2316 [==============================] - 6s - loss: 1.6930 - acc: 0.4227 - val_loss: 1.5279 - val_acc: 0.4495
Epoch 2/50
2316/2316 [==============================] - 5s - loss: 1.5218 - acc: 0.4495 - val_loss: 1.5386 - val_acc: 0.4495
Epoch 3/50
2316/2316 [==============================] - 5s - loss: 1.5270 - acc: 0.4495 - val_loss: 1.5129 - val_acc: 0.4495
Epoch 4/50
2316/2316 [==============================] - 5s - loss: 1.5126 - acc: 0.4495 - val_loss: 1.5086 - val_acc: 0.4495
Epoch 5/50
2316/2316 [==============================] - 5s - loss: 1.5081 - acc: 0.4495 - val_loss: 1.5050 - val_acc: 0.4495
Epoch 6/50
2316/2316 [==============================] - 5s - loss: 1.5048 - acc: 0.4495 - val_loss: 1.5029 - val_acc: 0.4495
Epoch 7/50
2316/2316 [==============================] - 5s - loss: 1.5005 - acc: 0.4495 - val_loss: 1.5007 - val_acc: 0.4495
Epoch 8/50
2316/2316 [==============================] - 5s - loss: 1.4954 - ac